## 자동 미분, autograd 

``autograd`` 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공한다.
이는 실행-기반-정의(define-by-run) 프레임워크로, 이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라진다.  

* autograd는 PyTorch에서 핵심적인 기능을 담당하는 하부 패키지이다.  
* autograd는 텐서의 연산에 대해 자동으로 미분값을 구해주는 기능을 한다.
* 텐서 자료를 생성할 때, `requires_grad`인수를 `True`로 설정하거나 `.requires_grad_(True)`를 실행하면 그 텐서에 행해지는 모든 연산에 대한 미분값을 계산한다.   
* 계산을 멈추고 싶으면 `.detach()`함수를 이용하면 된다. 


예제를 통해 알아 보도록 하자. `requires_grad`인수를 `True`로 설정하여  Tensor를 생성했다. 

In [1]:
import torch

In [2]:
x = torch.rand(2, 2, requires_grad=True)
print(x)

tensor([[0.9863, 0.9045],
        [0.5759, 0.9950]], requires_grad=True)


다음으로 이 x에 연산을 수행한다. 다음 코드의 y는 연산의 결과이므로 미분 함수를 가진다. `grad_fn`속성을 출력해 미분 함수를 확인 할 수 있다. 

In [3]:
y = torch.sum(x * 3)
print(y, y.grad_fn)

tensor(10.3852, grad_fn=<SumBackward0>) <SumBackward0 object at 0x0000017472511040>


`y.backward()` 함수를 실행하면 x의 미분값이 자동으로 갱신된다. x의 `grad`속성을 확인하면 미분값이 들어 있는 것을 확인 할 수 있다. y를 구하기 위한 x의 연산을 수식으로 쓰면 다음과 같다. 

$$
y = \displaystyle\sum_{i=1}^4 3 \times x_i
$$

이를 $x_i$에 대해 미분 하면 미분 함수는 다음과 같다. 

$$
\dfrac{\partial y}{\partial x_i} = 3
$$

실제 미분값과 같은지 확인해보자.

In [5]:
print(x.grad)

None


In [6]:
y.backward() # gradient를 계산한다.
x.grad

tensor([[3., 3.],
        [3., 3.]])

In [6]:
# y2 = torch.sum(x*4)
# y2.backward()
# x.grad

tensor([[7., 7.],
        [7., 7.]])

In [7]:
# y2 = torch.sum(x*4)
# x.grad.zero_()
# y2.backward()
# x.grad

tensor([[4., 4.],
        [4., 4.]])

### retain_graph 

- retain_graph = False가 기본이다.
- retain_graph = True ratain 이라는 말에서 알 수 있듯이 텐서들의 연속된 연산으로 구성되는 graph 를 유지한다. 
- retain_graph = False 이면 두 번째 backward를 시행시 저장되어 있던 중간 결과값들이 free 되었다고 나오면서 retain_graph=True 로 설정하라고 나온다. 
- Pytorch 에서는 backward() 메소드를 한 번 수행하면 기울기가 계산되면서 이를 계산하기 위한 중간 결과값들이 없어진다. 따라서 backward를 반복적으로 수행할 경우 retain_graph=True 로 수행한다.

In [7]:
y.backward() # 2번째 backward() 호출시 retain_graph=False 로 에러가 난다.
x.grad

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [8]:
x = torch.rand(2, 2, requires_grad=True)
y = torch.sum(x * 3)

y.backward(retain_graph=True) # retain_graph = True

In [9]:
x.grad

tensor([[3., 3.],
        [3., 3.]])

In [10]:
y.backward(retain_graph=True) # grad는 누적된다.
x.grad

tensor([[6., 6.],
        [6., 6.]])

* ``.backward()`` 를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있다.  
* 모든 연산 과정을 encode 하여 순환하지 않는 그래프(acyclic graph)를 생성한다.  
* 각 tensor는 ``.grad_fn`` 속성을 갖고 있는데, 이는 ``Tensor`` 를 생성한 ``Function`` 을 참조하고 있다. 

* `backward()`함수는 자동으로 미분값을 계산해 `requires_grad`인수가 True로 설정된 변수의 `grad`속성의 값을 갱신한다.
* 미분값을 그대로 출력받아 사용하고 싶은 경우에는 `torch.autograd.grad()`함수에 출력값과 입력값을 입력하면 미분값을 출력한다. 

In [11]:
torch.autograd.grad(y, x)

(tensor([[3., 3.],
         [3., 3.]]),)

상황에 따라 특정 연산에는 미분값을 계산하고 싶지 않은 경우에는 `.detach()`함수를 사용한다. 예를 들어, 이전 코드의 결과 값 `y`에 로지스틱 함수 연산을 수행하고 이에 대한 미분 값을 계산 하고 싶지 않은 경우에 다음처럼 할 수 있다. 

In [12]:
y_1 = y.detach()
torch.sigmoid(y_1)

tensor(0.9508)

또한 ``with torch.no_grad():`` 로 코드 블럭을 감싸서 autograd가  
``.requires_grad=True`` 인 Tensor들의 연산 기록을 추적하는 것을 멈출 수 있다.

In [13]:
x = torch.randn(3, requires_grad=True)

print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


### 2차 미분 수행하기, second derivative

In [14]:
x = torch.tensor(1., requires_grad = True)
y = 2*x**3 + 5*x**2 + 8

In [15]:
first_derivative = torch.autograd.grad(y, x, 
                                       retain_graph=True, 
                                       create_graph=True)
second_derivative = torch. autograd.grad(first_derivative, x)

In [16]:
second_derivative

(tensor(22.),)

# Autograd 연습

autograd는 텐서의 연산에 대해 자동으로 미분값을 구해주는 기능을 한다. 텐서 자료를 생성할 때, requires_grad인수를 True로 설정하거나 .requires_grad_(True)로 설정한다.

In [17]:
import torch

1. 다음과 같은 텐서를 생성하시오.
```
x = torch.tensor(2.0, requires_grad=True)
y = 9*x**4 + 2*x**3 + 3*x**2 + 6*x +1
```

In [20]:
x = torch.tensor(2.0, requires_grad=True)
y = 9*x**4 + 2*x**3 + 3*x**2 + 6*x +1

2. y를 x에 대해 미분하고 그 값을 출력해 보시오. .backward()를 사용하면 자동으로 미분값을 계산한다.

In [19]:
first_derivative = torch.autograd.grad(y, x, 
                                       retain_graph=True, 
                                       create_graph=True)
print(first_derivative)

(tensor(330., grad_fn=<AddBackward0>),)


In [21]:
y.backward()

In [22]:
x.grad

tensor(330.)

3. 다음과 같은 텐서를 생성하시오.
```
x = torch.tensor(1.0, requires_grad=True)
z = torch.tensor(2.0, requires_grad=True)
y = x**2 + z**3
```

In [23]:
x = torch.tensor(1.0, requires_grad=True)
z = torch.tensor(2.0, requires_grad=True)
y = x**2 + z**3

4. 3번에서 생성한 텐서에서 y를 x에 대해 편미분한 값과 y를 z에 대해 편미분한 값을 출력하시오.

In [25]:
first_derivative_x = torch.autograd.grad(y, x, 
                                       retain_graph=True, 
                                       create_graph=True)
print(first_derivative_x)

first_derivative_z = torch.autograd.grad(y, z, 
                                       retain_graph=True, 
                                       create_graph=True)
print(first_derivative_z)
                                        

(tensor(2., grad_fn=<MulBackward0>),)
(tensor(12., grad_fn=<MulBackward0>),)


5. 3번에서 생성한 x의 값을 출력해 보시오. .data를 사용한다.

In [26]:
x.data

tensor(1.)